In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.cross_validation import train_test_split
import gc
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm
%matplotlib inline

tqdm.pandas()

from contextlib import contextmanager
@contextmanager
def timer(name):
    import time
    startTime = time.time()
    yield
    elapsedTime = time.time() - startTime
    print('[{}] finished in {} s'.format(name, int(elapsedTime)))

train = pd.read_csv('train.csv')
train_data = train[['file_id','label']].drop_duplicates()
train_data.head()

/home/admin/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


,file_id,label
0,0,0
424,1,5
426,2,5
460,3,5
3260,4,5


In [2]:
train.groupby(['file_id'])['tid'].quantile(0.2).values

array([2644. , 2524. , 2516. , ..., 2338.4, 2448. , 2668. ])

In [3]:
api_opt = ['count','nunique']
for opt in api_opt:
    tmp = train.groupby(['file_id'])['api'].agg({'fileid_api_' + opt: opt}).reset_index() 
    train_data = pd.merge(train_data,tmp,how='left', on='file_id')
tid_opt = ['count','nunique','max','min','median','std'] 

for opt in tid_opt:
    print(opt)
    tmp = train.groupby(['file_id'])['tid'].agg({'fileid_tid_' + opt: opt}).reset_index() 
    train_data = pd.merge(train_data,tmp,how='left', on='file_id')

secs = [0.2,0.4,0.6,0.8]
for sec in secs: 
    train_data['fileid_tid_quantile_' + str(sec * 100)] = train.groupby(['file_id'])['tid'].quantile(sec).values
train_data['fileid_tid_range'] = train.groupby(['file_id'])['tid'].quantile(0.975).values - train.groupby(['file_id'])['tid'].quantile(0.0125).values

index_opt = ['count','nunique','max','min','median','std'] 
for opt in index_opt:
    print(opt)
    tmp = train.groupby(['file_id'])['index'].agg({'fileid_index_' + opt: opt}).reset_index() 
    train_data = pd.merge(train_data,tmp,how='left', on='file_id')

/home/admin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


count


/home/admin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  if __name__ == '__main__':


nunique
max
min
median
std
count


/home/admin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version


nunique
max
min
median
std


In [4]:
all_apis = set(train['api'])
def api_num(x, api_name):
    x = x.split(' ')
    return x.count(api_name)

def api_rate_num(x, api_name):
    x = x.split(' ')
    return x.count(api_name) / len(x)

def apply_mul_core(df):
    import multiprocessing as mlp
    num_cpu = 12
    pool = mlp.Pool(num_cpu)
    batch_num = 1 + len(df) // num_cpu
    results = []
    for i in range(num_cpu):
        task = df[i*batch_num : (i+1)*batch_num]
        result = pool.apply_async(multi_task,(task,))
        results.append(result)
    pool.close()
    pool.join()
    res = pd.DataFrame({})
    for result in results:
        feat = result.get()
        res = pd.concat([res, feat])
    return res

temp = train.groupby(['file_id'])['api'].apply(lambda x:' '.join(list(x))).to_frame().reset_index()
train_data = pd.merge(train_data,temp,how='left', on='file_id')

all_apis = list(all_apis)

def multi_task(df):
    for name in tqdm(all_apis):
        df[f'api_{name}_num'] = df['api'].apply(api_num, api_name=name)
        df[f'api_{name}_rate_num'] = df['api'].apply(api_rate_num, api_name=name)
    return df

df = apply_mul_core(train_data)

 54%|█████▎    | 165/308 [33:05<28:03, 11.78s/it] Process ForkPoolWorker-5:
Process ForkPoolWorker-7:
Process ForkPoolWorker-10:
Process ForkPoolWorker-11:
Process ForkPoolWorker-2:
Process ForkPoolWorker-9:
Process ForkPoolWorker-4:
Process ForkPoolWorker-8:
Process ForkPoolWorker-12:
Process ForkPoolWorker-1:
Process ForkPoolWorker-6:
Process ForkPoolWorker-3:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/admin/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/admin/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    se

  File "/home/admin/anaconda3/lib/python3.6/site-packages/pandas/core/series.py", line 3192, in apply
    mapped = lib.map_infer(values, f, convert=convert_dtype)
  File "/home/admin/anaconda3/lib/python3.6/site-packages/pandas/core/series.py", line 3192, in apply
    mapped = lib.map_infer(values, f, convert=convert_dtype)
  File "/home/admin/anaconda3/lib/python3.6/site-packages/pandas/core/series.py", line 3192, in apply
    mapped = lib.map_infer(values, f, convert=convert_dtype)
  File "/home/admin/anaconda3/lib/python3.6/site-packages/pandas/core/series.py", line 3192, in apply
    mapped = lib.map_infer(values, f, convert=convert_dtype)
  File "/home/admin/anaconda3/lib/python3.6/site-packages/pandas/core/series.py", line 3192, in apply
    mapped = lib.map_infer(values, f, convert=convert_dtype)
  File "pandas/_libs/src/inference.pyx", line 1472, in pandas._libs.lib.map_infer
  File "pandas/_libs/src/inference.pyx", line 1472, in pandas._libs.lib.map_infer
  File "pandas/_libs/

KeyboardInterrupt: 

In [ ]:
train_X = df.drop(['file_id','label', 'api'],axis=1).values
train_Y = df['label'].values
from sklearn.cross_validation import StratifiedKFold
import lightgbm as lgb

skf = StratifiedKFold(train_Y, n_folds=5, shuffle=True, random_state=2018)
for i,(tr_idx,val_idx) in enumerate(skf):
        print('FOLD: ',i)
        X_train,X_train_label = train_X[tr_idx],train_Y[tr_idx]
        X_val,X_val_label = train_X[val_idx],train_Y[val_idx]
        dtrain = lgb.Dataset(X_train,X_train_label) 
        dval   = lgb.Dataset(X_val,X_val_label, reference = dtrain)   
        params = {
                'task':'train', 
                'boosting_type':'gbdt',
                'num_leaves': 15,
                'objective': 'multiclass',
                'num_class':8,
                'learning_rate': 0.01,
                'feature_fraction': 0.85,
                'subsample':0.85,
                'num_threads': 28,
                'metric':'multi_logloss',
                'seed':2018
            }
        model = lgb.train(params, 
                          dtrain, 
                          num_boost_round=100000,
                          valid_sets=[dval],
                          verbose_eval=100, 
                          early_stopping_rounds=100)

In [ ]:
train_data.head()

In [ ]:
from sklearn.feature_extraction.text  import TfidfVectorizer
import os 
import pickle
n_range=(1,4)  #
max_feature=100000 #

if os.path.exists("API-Tfidf_%s.pkl"%str(n_range))==False:
    print("xx")
    api_tfidf=TfidfVectorizer(ngram_range=n_range,max_features=max_feature,min_df=2,max_df=0.97)
    api_tfidf.fit(train_data['api'].values)
    with open("API-Tfidf_%s.pkl"%str(n_range),'wb') as f:
        pickle.dump(api_tfidf,f)
        
else:
    with open("API-Tfidf_%s.pkl"%str(n_range),'rb')  as f:
        api_tfidf=pickle.load(f)

train_x=api_tfidf.transform(train_data['api'].values)
#test_x=api_tfidf.transform(test_data['api'].values)

In [ ]:
train_x.shape,train_Y.shape

In [ ]:
train_X = train_x
train_Y = df['label'].values
from sklearn.cross_validation import StratifiedKFold
import lightgbm as lgb

skf = StratifiedKFold(train_Y, n_folds=5, shuffle=True, random_state=2018)
for i,(tr_idx,val_idx) in enumerate(skf):
        print('FOLD: ',i)
        X_train,X_train_label = train_X[tr_idx],train_Y[tr_idx]
        X_val,X_val_label = train_X[val_idx],train_Y[val_idx]
        dtrain = lgb.Dataset(X_train,X_train_label) 
        dval   = lgb.Dataset(X_val,X_val_label, reference = dtrain)   
        params = {
                'task':'train', 
                'boosting_type':'gbdt',
                'num_leaves': 15,
                'objective': 'multiclass',
                'num_class':8,
                'learning_rate': 0.01,
                'feature_fraction': 0.85,
                'subsample':0.85,
                'num_threads': 28,
                'metric':'multi_logloss',
                'seed':2018
            }
        model = lgb.train(params, 
                          dtrain, 
                          num_boost_round=100000,
                          valid_sets=[dval],
                          verbose_eval=100, 
                          early_stopping_rounds=100)